In [7]:
from bs4 import BeautifulSoup as bs
import pandas  as pd
import requests 
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine

url = 'https://jiji.ng/'
page = requests.get(url)
soup = bs(page.content, 'html5lib')
#print(soup.prettify())
table = soup.find('div', attrs= {'class': 'h-dflex h-flex-wrap h-mh--7 h-mb--15'})
#print(table)
rows = table.findAll('div', attrs={'class': 'b-listing-cards__item'})
items = []
df = pd.DataFrame(columns=['title', 'price', 'imageCount', 'link'])
for row in rows:
    item = {}
    item['title'] = row.find('div', attrs={'class': 'b-trending-card__title'}).text.replace("\n        ", "").replace("\n      ","")
    item['price'] = row.find('div', attrs={'class': 'b-trending-card__price'}).text.replace("\n        ", "").replace("\n      ","").replace("₦ ","")
    item['imageCount'] = row.find('div', attrs={'class': 'b-trending-card__counter'}).text.replace("\n        ", "").replace("\n      ","")
    item['link'] = row.a['href']
    
    df = df.append(item, ignore_index=True)
    #print(item)
#df.to_csv('jiji_trending_adds.csv')
try:
 conn = pymysql.connect(host='localhost', user='root', password='', database="univelcity", charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
 engine = create_engine('mysql://root:@localhost/univelcity')
 with engine.begin() as connection:
  df.to_sql(name='web_scrapping_data', con=connection, if_exists='append', index=False)
 print('Sucessfully written to Database!!!')

except Exception as e:
    print(e)


Sucessfully written to Database!!!


In [9]:
from bs4 import BeautifulSoup as bs
import pandas  as pd
import requests 
import pymysql

url = 'https://jiji.ng/'
page = requests.get(url)
soup = bs(page.content, 'html5lib')
#print(soup.prettify())
table = soup.find('div', attrs= {'class': 'h-dflex h-flex-wrap h-mh--7 h-mb--15'})
#print(table)
rows = table.findAll('div', attrs={'class': 'b-listing-cards__item'})
items = []
df = pd.DataFrame(columns=['title', 'price', 'imageCount', 'link'])
for row in rows:
    item = {}
    item['title'] = row.find('div', attrs={'class': 'b-trending-card__title'}).text.replace("\n        ", "").replace("\n      ","")
    item['price'] = row.find('div', attrs={'class': 'b-trending-card__price'}).text.replace("\n        ", "").replace("\n      ","").replace("₦ ","")
    item['imageCount'] = row.find('div', attrs={'class': 'b-trending-card__counter'}).text.replace("\n        ", "").replace("\n      ","")
    item['link'] = row.a['href']
    
    df = df.append(item, ignore_index=True)
    try:
        ##   Connect to database
        db = pymysql.connect(host='localhost', user='root', password='', database="univelcity", charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)

        ### prepare a cursor object using cursor() method
        cursor = db.cursor()
        sql = "INSERT INTO web_scrapping_data (title, price, imageCount, link) VALUES(%s, %s, %s, %s)"
        cursor.execute(sql, (item['title'], item['price'], item['imageCount'], item['link']))
        db.commit()
        print("Data inserted")
    except Exception as e:
        db.rollback()
        print(e)
    db.close()

df.to_csv('jiji_trending_adds.csv')
